# <b>Probabilidade

## Teorema de Bayes

O Teorema de Bayes descreve a probabilidade de um evento, baseada em informações prévias que podem estar relacionadas ao evento.

A fórmula do Teorema de Bayes é:

$$
P(A|B) = \frac{P(B|A) \cdot P(A)}{P(B)}
$$

Onde:

- $P(A|B)$: probabilidade de $A$ dado que $B$ ocorreu (probabilidade posterior);
- $P(B|A)$: probabilidade de $B$ dado que $A$ ocorreu (verossimilhança);
- $P(A)$: probabilidade de $A$ (probabilidade a priori);
- $P(B)$: probabilidade de $B$ (evidência total).

---

Se houver vários eventos $A_i$, a evidência $P(B)$ pode ser calculada como:

$$
P(B) = \sum_{i=1}^{n} P(B|A_i) \cdot P(A_i)
$$

Essa fórmula permite aplicar o teorema em contextos com múltiplas hipóteses.


In [141]:
def bayes_theorem(likelihood, prior, evidence):
    """
    Calcula a probabilidade posterior usando o Teorema de Bayes.
    
    Parâmetros:
    - likelihood: P(B|A), verossimilhança
    - prior: P(A), probabilidade a priori
    - evidence: P(B), evidência

    Retorna:
    - posterior: P(A|B), probabilidade posterior
    """
    if evidence == 0:
        raise ValueError("A evidência não pode ser zero.")
    
    posterior = (likelihood * prior) / evidence
    return posterior

# Exemplo de uso do Teorema de Bayes:
# 1% da população tem uma doença.
# O teste dá positivo em 99% dos casos com a doença e positivo em 5% dos casos sem a doença.

P_A = 0.01              # Probabilidade de ter a doença
P_B_given_A = 0.99      # Probabilidade do teste ser positivo dado que a pessoa tem a doença
P_B_given_not_A = 0.05  # Probabilidade do teste ser positivo dado que a pessoa NÃO tem a doença
P_not_A = 1 - P_A

# Evidência total
P_B = (P_B_given_A * P_A) + (P_B_given_not_A * P_not_A)

# Aplicando o Teorema de Bayes
posterior = bayes_theorem(P_B_given_A, P_A, P_B)
print(f"Probabilidade de ter a doença dado teste positivo: {posterior:.4f}")

Probabilidade de ter a doença dado teste positivo: 0.1667


## Função de Distribuição Acumulada (FDA ou CDF)

A função de distribuição acumulada (FDA), também chamada de CDF (do inglês *Cumulative Distribution Function*), descreve a **probabilidade de que uma variável aleatória $X$ assuma um valor menor ou igual a um determinado valor $x$**.

$$
F(x) = P(X \leq x)
$$

No caso de dados observados (dados empíricos), a distribuição acumulada empírica é dada por:

$$
\hat{F}(x) = \frac{\text{número de elementos } \leq x}{\text{número total de elementos}}
$$

Essa função é **monotonicamente crescente**, e seu valor está sempre entre 0 e 1.

In [142]:
def distribuicao_acumulada(conjunto_de_dados, x):
    """
    Calcula a função de distribuição acumulada empírica (ECDF) para um valor x.

    Parâmetros:
    - conjunto_de_dados: lista ou array com os dados numéricos
    - x: valor até o qual se deseja calcular a proporção acumulada

    Retorna:
    - F_x: proporção dos valores no conjunto que são menores ou iguais a x

    Fórmula:
    F(x) = número de valores <= x / número total de valores
    """
    n = len(conjunto_de_dados)
    count = sum(1 for i in conjunto_de_dados if i <= x)
    F_x = count / n
    return F_x

# Exemplo de uso
amostra = [45, 56, 67, 70, 75, 80, 83, 88, 90, 95]
x = 75
print(f"A proporção de valores <= {x} é {distribuicao_acumulada(amostra, x)}")

A proporção de valores <= 75 é 0.5


## Função Densidade de Probabilidade (FDP ou PDF)

A função densidade de probabilidade (FDP), conhecida como PDF (*Probability Density Function*), descreve a **probabilidade relativa** de uma variável aleatória contínua assumir um determinado valor.

Para uma variável contínua $X$ com distribuição normal, a PDF é definida por:

$$
f(x) = \frac{1}{\sigma \sqrt{2\pi}} \cdot e^{ -\frac{1}{2} \left( \frac{x - \mu}{\sigma} \right)^2 }
$$

Onde:

- $\mu$ é a média da distribuição
- $\sigma$ é o desvio padrão
- $e$ é a base do logaritmo natural

---

### Interpretação da PDF

Para variáveis contínuas, a probabilidade de um valor exato ocorrer é **zero**:

$$
P(X = x) = 0
$$

O que realmente interessa é a **probabilidade de um intervalo**, que pode ser obtida calculando a área sob a curva da PDF entre dois pontos $a$ e $b$:

$$
P(a \leq X \leq b) = \int_a^b f(x)\,dx
$$

---

### Uso da Regra dos Trapézios para resolver a Integral

A ideia da regra dos trapézios é:

- Dividir o intervalo $[a, b]$ em $n$ partes pequenas;
- Calcular a área de cada trapézio sob a curva da PDF;
- Somar todas essas áreas para estimar a integral:

$$
\int_a^b f(x)\,dx \approx \sum_{i=1}^{n} \frac{f(x_i) + f(x_{i+1})}{2} \cdot (x_{i+1} - x_i)
$$

Quanto maior o número de subdivisões, mais precisa será a estimativa da área — e, portanto, da **probabilidade acumulada no intervalo**.

In [143]:
def densidade_normal(x, media=0, desvio=1):
    """
    Calcula a função densidade de probabilidade (PDF) da distribuição normal.

    Parâmetros:
    - x: valor da variável aleatória
    - media: média da distribuição (μ)
    - desvio: desvio padrão da distribuição (σ)

    Retorna:
    - f_x: valor da PDF em x
    """
    pi = 3.141592653589793
    e = 2.718281828459045

    coeficiente = 1 / (desvio * (2 * pi) ** 0.5)
    expoente = -0.5 * ((x - media) / desvio) ** 2
    f_x = coeficiente * (e ** expoente)
    return f_x

def probabilidade_intervalo(a, b, media=0, desvio=1, passos=1000):
    """
    Estima a probabilidade de P(a <= X <= b) para uma distribuição normal,
    usando a regra dos trapézios para integração numérica.

    Parâmetros:
    - a: limite inferior do intervalo
    - b: limite superior do intervalo
    - media: média da distribuição
    - desvio: desvio padrão
    - passos: número de subdivisões (maior = mais preciso)

    Retorna:
    - estimativa da probabilidade no intervalo [a, b]
    """
    largura = (b - a) / passos
    soma = 0
    for i in range(passos):
        x0 = a + i * largura
        x1 = a + (i + 1) * largura
        y0 = densidade_normal(x0, media, desvio)
        y1 = densidade_normal(x1, media, desvio)
        area_trapezio = (y0 + y1) * largura / 2
        soma += area_trapezio
    return soma

# Estimando P(60 <= X <= 80) em uma normal com média 70 e desvio padrão 10
p = probabilidade_intervalo(60, 80, media=70, desvio=10)
print(f"Probabilidade entre 60 e 80: {p:.4f}")

Probabilidade entre 60 e 80: 0.6827


## Distribuição Binomial

A **Distribuição Binomial** descreve a **probabilidade de obter um número fixo de sucessos** em um número fixo de tentativas, onde:

- Cada tentativa é independente;
- Só há dois resultados possíveis: **sucesso** ou **fracasso**;
- A probabilidade de sucesso $p$ é constante em cada tentativa.

---

### Fórmula da Probabilidade Binomial

A fórmula da distribuição binomial é:

$$
P(X = k) = \binom{n}{k} \cdot p^k \cdot (1 - p)^{n - k}
$$

Onde:

- $n$: número de tentativas  
- $k$: número de sucessos desejados  
- $p$: probabilidade de sucesso  
- $\binom{n}{k}$: coeficiente binomial (ou “n escolhe k”):

$$
\binom{n}{k} = \frac{n!}{k!(n - k)!}
$$

---

### Exemplo:

Se jogarmos uma moeda 5 vezes ($n = 5$), e a chance de dar cara for 0.5 ($p = 0.5$), a probabilidade de sair exatamente 3 caras ($k = 3$) é:

$$
P(X = 3) = \binom{5}{3} \cdot 0.5^3 \cdot 0.5^2 = 10 \cdot 0.125 \cdot 0.25 = 0.3125
$$

---

### Distribuição Acumulada Binomial (FDA)

A **função de distribuição acumulada binomial** nos dá a **probabilidade de obter até um certo número de sucessos** $k$ em $n$ tentativas:

$$
P(X \leq k) = \sum_{i=0}^{k} \binom{n}{i} p^i (1 - p)^{n - i}
$$

---

### Exemplo:

Em um teste com 10 perguntas, se a chance de acertar cada uma ao acaso for 0.2, qual a probabilidade de acertar **no máximo 3**?

$$
P(X \leq 3) = P(X=0) + P(X=1) + P(X=2) + P(X=3) = 0.8791
$$

Essa soma pode ser feita usando a **função acumulada** diretamente.

---

### Probabilidade de Pelo Menos $k$ Sucessos (P(X ≥ k))

A probabilidade de obter **pelo menos** $k$ sucessos em uma distribuição binomial é dada por:

$$
P(X \geq k) = 1 - P(X < k) = 1 - P(X \leq k - 1)
$$

Ou seja, subtrai-se da unidade a probabilidade acumulada de **menos de $k$ sucessos**.

---

### Exemplo:

Qual a probabilidade de acertar **pelo menos 3** questões ao acaso em um teste com 10 questões, sabendo que a chance de acerto por questão é 0.2?

$$
P(X \geq 3) = 1 - P(X \leq 2) = 0.1208
$$

---

**Resumo**:
- Use a **função binomial** para calcular $P(X = k)$.
- Use a **função acumulada** para calcular $P(X \leq k)$.
- Use o **complemento da acumulada** para calcular $P(X \geq k)$.


In [144]:
def fatorial(n):
    resultado = 1
    for i in range(2, n + 1):
        resultado *= i
    return resultado

def binomial_p(k, n, p):
    """
    Calcula a probabilidade de exatamente k sucessos em n tentativas com probabilidade p de sucesso.

    Parâmetros:
    - n: número total de tentativas
    - k: número de sucessos desejados
    - p: probabilidade de sucesso

    Retorna:
    - Probabilidade de obter k sucessos
    """
    # Coeficiente binomial: C(n, k)
    c = fatorial(n) / (fatorial(k) * fatorial(n - k))
    # Probabilidade
    prob = c * (p ** k) * ((1 - p) ** (n - k))
    return prob

def binomial_acumulada(k_max, n, p):
    """
    Soma das probabilidades de 0 até k_max sucessos em n tentativas.

    Parâmetros:
    - n: número total de tentativas
    - k_max: número máximo de sucessos (inclusive)
    - p: probabilidade de sucesso

    Retorna:
    - P(X <= k_max)
    """
    soma = 0
    for k in range(k_max + 1):
        soma += binomial_p(k, n, p)
    return soma

def binomial_maior_igual(k_min, n, p):
    """
    Calcula P(X >= k_min) para uma variável binomial.

    Parâmetros:
    - n: número total de tentativas
    - k_min: número mínimo de sucessos desejado
    - p: probabilidade de sucesso

    Retorna:
    - P(X >= k_min)
    """
    if k_min <= 0:
        return 1.0  # Toda a distribuição está inclusa
    return 1 - binomial_acumulada(k_min, n, p)

# Exemplo:
# Sabendo que uma caixa com 12 ovos possui a probabilidade de 5% de um dos ovos ser quebrados
# Iremos estimas as probabilidades:
k, n, p = 2, 12, 0.05

print(f"Exatamente {k} ovos estarem quebrados: {binomial(k=k, n=n, p=p)}")
print(f"Ter no máximo {k} ovos quebrados: {binomial_acumulada(k_max=k, n=n, p=p)}")
print(f"Ter mais de {k} ovos quebrados: {binomial_maior_igual(k_min=k, n=n, p=p)}")

Exatamente 2 ovos estarem quebrados: 0.0987915949743325
Ter no máximo 2 ovos quebrados: 0.9804317380028451
Ter mais de 2 ovos quebrados: 0.01956826199715489


## Distribuição de Poisson

A **Distribuição de Poisson** descreve a **probabilidade de um número fixo de eventos ocorrer** em um intervalo de tempo ou espaço, sob as seguintes condições:

- Os eventos ocorrem de forma **independente**;
- A taxa média ($\lambda$) de ocorrência é **constante**;
- Dois eventos **não ocorrem ao mesmo tempo**.

---

### Fórmula da Distribuição de Poisson

A fórmula para calcular a probabilidade de exatamente $k$ ocorrências é:

$$
P(X = k) = \frac{e^{-\lambda} \cdot \lambda^k}{k!}
$$

Onde:

- $k$: número de ocorrências desejadas  
- $\lambda$: média de ocorrências no intervalo  
- $e$: constante de Euler (aproximadamente 2.718)

---

### Exemplo:

Se em média ocorrem 4 chamadas por minuto em um call center ($\lambda = 4$), qual a probabilidade de receber **exatamente 2 chamadas** em um determinado minuto?

$$
P(X = 2) = \frac{e^{-4} \cdot 4^2}{2!} = \frac{e^{-4} \cdot 16}{2} \approx 0.1465
$$

---

### Distribuição Acumulada de Poisson (FDA)

A **função de distribuição acumulada de Poisson** calcula a **probabilidade de até $k$ ocorrências**:

$$
P(X \leq k) = \sum_{i = 0}^{k} \frac{e^{-\lambda} \cdot \lambda^i}{i!}
$$

---

### Exemplo:

Se $\lambda = 3$, qual a probabilidade de ocorrerem **no máximo 2 eventos**?

$$
P(X \leq 2) = P(X = 0) + P(X = 1) + P(X = 2)
$$

---

### Probabilidade de Pelo Menos $k$ Ocorrências (P(X ≥ k))

A probabilidade de ocorrerem **pelo menos $k$ eventos** é dada por:

$$
P(X \geq k) = 1 - P(X < k) = 1 - P(X \leq k - 1)
$$

---

### Exemplo:

Se $\lambda = 5$, qual a probabilidade de ocorrerem **pelo menos 3 eventos**?

$$
P(X \geq 3) = 1 - P(X \leq 2)
$$

---

**Resumo**:
- Use a **função de Poisson** para calcular $P(X = k)$.
- Use a **função acumulada** para calcular $P(X \leq k)$.
- Use o **complemento da acumulada** para calcular $P(X \geq k)$.

In [146]:
def fatorial(n):
    if n == 0 or n == 1:
        return 1
    resultado = 1
    for i in range(2, n + 1):
        resultado *= i
    return resultado

def poisson(k, lam):
    """
    Calcula a probabilidade de exatamente k eventos em uma distribuição de Poisson.

    Parâmetros:
    - k: número de eventos desejado (int)
    - lam: taxa média de eventos (float)

    Retorna:
    - Probabilidade P(X = k)
    """
    e = 2.718281828459045  # valor aproximado de e
    prob = (e ** (-lam)) * (lam ** k) / fatorial(k)
    return prob

def poisson_cdf(k, lam):
    """
    Calcula a probabilidade acumulada até k (P(X <= k)) em uma distribuição de Poisson.

    Parâmetros:
    - k: valor máximo de eventos (int)
    - lam: taxa média de eventos (float)

    Retorna:
    - Probabilidade acumulada
    """
    soma = 0
    for i in range(0, k + 1):
        soma += poisson(i, lam)
    return soma

def poisson_geq(k, lam):
    """
    Calcula a probabilidade de pelo menos k eventos (P(X > k)).

    Parâmetros:
    - k: número mínimo de eventos (int)
    - lam: taxa média de eventos (float)

    Retorna:
    - Probabilidade complementar
    """
    if k == 0:
        return 1.0
    return 1 - poisson_cdf(k, lam)

# λ = Média ou Variância na distribuição Poisson
k, lam = 7, 6

print("P(X = k):", poisson(k, lam))
print("P(X <= k):", poisson_cdf(k, lam))
print("P(X > k):", poisson_geq(k, lam))

P(X = k): 0.13767697804112577
P(X <= k): 0.7439797604537173
P(X > k): 0.2560202395462827
